In [31]:
import os
from yt_dlp import YoutubeDL
import cv2
import imutils
import requests
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from vidgear.gears import CamGear
from imutils.object_detection import non_max_suppression
from imutils import paths

In [3]:
print(tf.__version__)  # 2.13.0-rc2
print(tf.config.list_physical_devices('GPU')) # [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
print(tf.test.is_built_with_cuda)  # <function is_built_with_cuda at 0x169361d00>
print(tf.test.gpu_device_name())  # /device:GPU:0
print(tf.config.get_visible_devices())  # [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2.13.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
<function is_built_with_cuda at 0x7f6e869873a0>
/device:GPU:0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-08-21 10:36:42.330816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 10:36:42.545229: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 10:36:42.545379: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
# Download model if not already downloaded

if not os.path.isdir('mobilenet_v2_140_224'):
    if not os.path.isfile('mobilenet_v2_140_224.1.zip'):
    !wget https://github.com/GantMan/nsfw_model/releases/download/1.2.0/mobilenet_v2_140_224.1.zip
    !unzip mobilenet_v2_140_224.1.zip

IndentationError: expected an indented block (674254128.py, line 5)

In [4]:
# Load model
model = tf.keras.models.load_model('mobilenet_v2_140_224')
# Test model
model.predict(np.zeros((1, 224, 224, 3)))  # array([[0.001, 0.999]], dtype=float32)

2023-08-21 10:36:45.782012: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 10:36:45.782184: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 10:36:45.782255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

2023-08-21 10:36:48.247227: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


1/1 [==============================] - 1s 925ms/step


2023-08-21 10:36:48.659214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([[0.28864017, 0.20069636, 0.48946753, 0.01522448, 0.00597143]],
      dtype=float32)

In [5]:
# Get labels
labels = []
with open('mobilenet_v2_140_224/class_labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]
for l in labels:
    print(l)

tf.debugging.set_log_device_placement(True)
output = model(tf.random.uniform([1, 224, 224, 3]))
print(output)

drawings
hentai
neutral
porn
sexy
tf.Tensor([[7.6401830e-02 5.9221252e-03 9.1742486e-01 1.7604571e-04 7.5179407e-05]], shape=(1, 5), dtype=float32)


In [9]:
# Find GStreamer version
buildInfo = cv2.getBuildInformation()
SUPPORT_GSTREAMER = True
for line in buildInfo.split('\n'):
    if 'GStreamer' in line:
        print(line)
        if "NO" in line:
            SUPPORT_GSTREAMER = False
        

    GStreamer:                   NO


In [36]:
youtube_video_id = "thJgU9jkdU4"
url = "https://youtu.be/" + youtube_video_id
url = ""
source = None
filename_ydl = None
if not SUPPORT_GSTREAMER:
    # Download instead of streaming
    if "youtu" in url:
        with YoutubeDL() as ydl:
            ydl.download([url])
            info = ydl.extract_info(url, download=False)
            # Get file path
            # filename_ydl = ydl.prepare_filename(info)
            # print(filename_ydl)
    else:
        # Download video
        filename_ydl = url.split('/')[-1]
        resp = requests.get(url, stream=True)
        with open(filename_ydl, 'wb') as f:
            for chunk in resp.iter_content(chunk_size=1024):
                f.write(chunk)
    
    

In [11]:
# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [40]:
# infinite loop
if not SUPPORT_GSTREAMER:
    source = CamGear(source=filename_ydl, logging=True)
else:
    source = CamGear(source=url, stream_mode = True, logging=True)
stream = source.start() # YouTube Video URL as input
# Print resolution of stream
frame = stream.read()
if frame is None: 
    raise Exception("Unable to read stream from source")
print(frame.shape)
# How many 224x224 square you can fit in the frame
print(frame.shape[0] // 224, frame.shape[1] // 224)
# How many pixels are left over
leftoverX = frame.shape[0] % 224
leftoverY = frame.shape[1] % 224
# Offset to center the squares based on leftover pixels
offsetX = (leftoverX // 2) + (leftoverX % 2)
offsetY = (leftoverY // 2) + (leftoverY % 2)
print(offsetX, offsetY)
labelsToColor = {
    "drawings": (0, 0, 0),
    "hentai": (0, 0, 255),
    "neutral": (0, 0, 0),
    "porn": (0, 255, 0),
    "sexy": (255, 0, 0)
}
while True:
    
    frame = stream.read()
    # read frames

    # check if frame is None
    if frame is None:
        #if True break the infinite loop
        break
    
    # do something with frame here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect people in the image
    #(rects, weights) = hog.detectMultiScale(gray, winStride=(4, 4), padding=(8, 8), scale=1.05)
    #pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    
    #for (x, y, w, h) in pick:
    #    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    # Divide frame into 224x224 squares by drawing rectangles
    # For each square, predict if it is NSFW
    # If NSFW, blur the square
    batchOfBatch = []
    for i in range(frame.shape[0] // 224):
        batch = []
        for j in range(frame.shape[1] // 224):
            square_points = [(j * 224 + offsetX, i * 224 + offsetY), (j * 224 + 224 + offsetX, i * 224 + 224 + offsetY)]
            # Crop square
            square = frame[square_points[0][1]:square_points[1][1], square_points[0][0]:square_points[1][0]]
            
            # Write square to file
            #cv2.imwrite("square.jpg", square)
            # Resize square to 224x224 
            square = cv2.resize(square, (224, 224))
            # Add square to batch
            batchOfBatch.append(square)
            
    
    batch = np.array(batchOfBatch)
    
    predictions = model.predict(batch)
    
    for i in range(frame.shape[0] // 224):
        indexBatch = 0
        # Predict NSFW for batch
        # Blur NSFW squares
        for j in range(frame.shape[1] // 224):
            prediction = predictions[indexBatch]
            square_points = [(j * 224 + offsetX, i * 224 + offsetY), (j * 224 + 224 + offsetX, i * 224 + 224 + offsetY)]
            # Write prediction on frame
            for k in range(5):
                text = labels[k] + ": " + str(round(prediction[k], 2))
                color = labelsToColor[labels[k]]
                cv2.rectangle(frame, (j * 224 + offsetX, i * 224 + offsetY + 20 * k), (j * 224 + offsetX + int(prediction[k] * 100), i * 224 + offsetY + 20 * (k + 1)), color, -1)
                cv2.putText(frame, text, (j * 224 + offsetX, i * 224 + offsetY + 25 * k), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.rectangle(frame, square_points[0], square_points[1], (0, 0, 255), 2)
            if prediction[1] > 0.5:
                frame[i * 224 + offsetY:i * 224 + 224 + offsetY, j * 224 + offsetX:j * 224 + 224 + offsetX] = cv2.blur(frame[i * 224 + offsetY:i * 224 + 224 + offsetY, j * 224 + offsetX:j * 224 + 224 + offsetX], (50, 50))
            indexBatch += 1
    
        
    cv2.imshow("Output Frame", frame)
    # Show output window

    key = cv2.waitKey(1) & 0xFF
    # check for 'q' key-press
    if key == ord("q"):
        #if 'q' key-pressed break out
        break

cv2.destroyAllWindows()
# close output window

# safely close video stream.
stream.stop()

11:07:10 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!


(480, 854, 3)
2 3
16 91
1/1 [==============================] - 0s 17ms/step


11:07:30 ::    CamGear    ::  DEBUG   :: Terminating processes.
